In [7]:
import cv2
import numpy as np

In [8]:
cam = cv2.VideoCapture(0)
name = input("Enter The Name: ")
data_path = "./data/"
model = cv2.CascadeClassifier("front_face.xml")
facedata=[]
offset = 20
skip = 0

while True:
    success , img = cam.read()
    if not success :
        print("Failed")
    grayimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = model.detectMultiScale(img,1.3,1)
    faces = sorted(faces,key=lambda f: f[2]*f[3])
    if len(faces)>0:
        
        f = faces[-1]
        x,y,w,h = f
        cv2.rectangle(img , (x,y),(x+w,y+h),(0,255,0),2)
        cropped_img = img[y-offset:y+h+offset,x-offset:x+w+offset]
        cropped_img = cv2.resize(cropped_img,(100,100))
        skip = skip + 1
        
    if skip % 10 == 0 :
        facedata.append(cropped_img)
        print(len(facedata))
        
    cv2.imshow("image",img)
    key = cv2.waitKey(1)
    
    if key==ord('q'):
        break

facedata = np.asarray(facedata)
m = facedata.shape[0]
facedata = facedata.reshape((m,-1))
file = data_path + name + ".npy"
np.save(file,facedata)
cam.release()
cv2.destroyAllWindows()
    

Enter The Name: PrinceAman
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
2

In [14]:
df = np.load("./data/PrinceAman.npy")

In [15]:
df

array([[100, 107, 112, ...,  37,  43,  43],
       [119, 122, 146, ...,  65,  61,  68],
       [144, 140, 143, ...,  78,  74,  83],
       ...,
       [ 44,  53,  79, ...,  83,  81,  88],
       [ 42,  60,  86, ..., 109,  87,  88],
       [ 56,  69, 114, ..., 151, 102,  81]], dtype=uint8)

In [16]:
df.shape

(504, 30000)

In [17]:
import os

In [18]:
data_path = "./data/"
FaceData = []
labels = []
mapping = {}
classid = 0

for f in os.listdir(data_path):
    if f.endswith(".npy"):
        mapping[classid] = f[:-4] 
        dataitem = np.load(data_path+f)
        FaceData.append(dataitem)
        m = dataitem.shape[0]
        target = classid*np.ones((m,))
        labels.append(target)
        
X = np.concatenate(FaceData , axis=0)
y = np.concatenate(labels , axis = 0).reshape((-1,1))


print(X.shape)
print(y.shape)
print(mapping)
        


(504, 30000)
(504, 1)
{0: 'princeaman'}


In [19]:
def dist(p,q):
    return np.sqrt(np.sum(p-q)**2)

def knn(X,y,xt,k=5):
    m = X.shape[0]
    dlist = []
    for i in range(m):
        d = dist(X[i],xt)
        dlist.append((d,y[i]))
        
    dlist = sorted(dlist)
    dlist = np.array(dlist[:k])
    labels = dlist[:,1]
    
    labels , cnts = np.unique(labels,return_counts=True)
    idx = cnts.argmax()
    pred = labels[idx]
    
    return int(pred)

In [20]:
cam = cv2.VideoCapture(0)
model = cv2.CascadeClassifier("front_face.xml")
offset = 20

while True:
    success , img = cam.read()
    if not success:
        print("Failed")
    faces = model.detectMultiScale(img,1.3,5)
    for f in faces:
        x , Y , w , h = f
        cv2.rectangle(img,(x,Y) , (x+w,Y+h),(0,255,0),0)
        cropped_img = img[Y-offset:Y + h + offset , x-offset:x+w+offset]
        cropped_img = cv2.resize(cropped_img,(100,100))
        classpredicted = knn(X,y,cropped_img.flatten())
        namepredicted = mapping[classpredicted]
        
        org = (x,Y-10)
        fontFace = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 1.5
        color = (0, 255, 0)
        thickness = 2
        lineType = cv2.LINE_AA
        cv2.putText(img,namepredicted , org, fontFace, fontScale, color, thickness, lineType)

        
    cv2.imshow("Image" , img)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
        
cam.release()
cv2.destroyAllWindows()
    
    

C:\Users\Hp\AppData\Local\Temp\ipykernel_28144\3601122786.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dlist = np.array(dlist[:k])
